<a href="https://colab.research.google.com/github/Aikoin/Fine_Tuning_BERT_for_Text_Classification/blob/main/mrpc%E5%90%8C%E4%B9%89%E5%88%A4%E6%96%AD1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The fine-tuned model should be able to distinguish if any two sentences are semantically equivalent.  
![](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*bqSEv_c5rzt3N__IQcudWg.png)

In [ ]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 3.7 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
from torch.utils.data import DataLoader

In [ ]:
# Use bert-base-uncased and prepare Dataset mrpc with padding
raw_dataset = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint) # AutoClasses are here to do this job for you so that you automatically retrieve the relevant model given the name/path to the pretrained weights/config/vocabulary.

In [ ]:
def tokenize_function(example):
  return tokenizer(
      example["sentence1"],
      example["sentence2"],
      truncation=True
      ) # This will truncate token by token, removing a token from the longest sequence in the pair until the proper length is reached.

In [ ]:
tokenized_datasets = raw_dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [ ]:
# Check what is inside raw_dataset
raw_train_dataset = raw_dataset["train"]
raw_train_dataset[0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

In [ ]:
# Preprocess datasets
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [ ]:
# Split datasets
train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle = True,
    batch_size = 8,
    collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"],
    batch_size = 8,
    collate_fn=data_collator
)

In [ ]:
# Check there is no mistake in the data processing
for batch in train_dataloader:
  break
{k: v.shape for k, v in batch.items()}

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 67]),
 'token_type_ids': torch.Size([8, 67]),
 'attention_mask': torch.Size([8, 67])}

In [ ]:
# Load model
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels =2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Check loss
output = model(**batch)
print(output.loss, output.logits.shape)

tensor(0.9239, grad_fn=<NllLossBackward0>) torch.Size([8, 2])


In [ ]:
# Define training parameter
from transformers import AdamW, get_scheduler
import torch

optimizer = AdamW(model.parameters(), lr = 5e-5)
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer = optimizer,
    num_warmup_steps = 0,
    num_training_steps = num_training_steps,
)

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [ ]:
# Train model
from tqdm.auto import tqdm
progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
  for batch in train_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    output = model(**batch)
    loss = output.loss
    loss.backward()

    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    progress_bar.update(1)

  0%|          | 0/1377 [00:00<?, ?it/s]

In [ ]:
# Show metric
import evaluate
metric = evaluate.load("glue", "mrpc")

model.eval()
for batch in eval_dataloader:
  batch = {k: v.to(device) for k, v in batch.items()}
  with torch.no_grad():
    output = model(**batch)

  logits = output.logits
  pred = torch.argmax(logits, dim = -1)
  metric.add_batch(predictions = pred, references = batch['labels'])

metric.compute()

{'accuracy': 0.875, 'f1': 0.911917098445596}

In [ ]:
# Mount google driver
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/NLP

Mounted at /content/drive
/content/drive/MyDrive/NLP


In [ ]:
# Save tokenizer and model to google driver
trained_model_dir = '/content/drive/MyDrive/NLP/model/bert-base-uncased_mrpc'
tokenizer.save_pretrained(trained_model_dir)
model.save_pretrained(trained_model_dir)